<a href="https://colab.research.google.com/github/KenzaxTazi/Agri-Risk/blob/master/Model_exploration_Raghul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm==4.42.1


In [0]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future

!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import xarray as xr
import numpy as np


In [0]:
import os, sys

path = '/content/drive/My Drive/Team Plants/data'
os.chdir(path)

# Import full dataset # 

Here, we import the below dataset

In [0]:
dataset = pd.read_csv("head_of_soils_recommendations_MGM.csv")

In [0]:
dataset

,Unnamed: 0,Unnamed: 0.1,alloc_key,x,y,iso3,maize_a_2010,maize_a_2005,nearest_lat,nearest_lon,CDD-Q1-2010,CDD-Q2-2010,CDD-Q3-2010,CDD-Q4-2010,CDD-Q1-2005,CDD-Q2-2005,CDD-Q3-2005,CDD-Q4-2005,CFD-Q1-2010,CFD-Q2-2010,CFD-Q3-2010,CFD-Q4-2010,CFD-Q1-2005,CFD-Q2-2005,CFD-Q3-2005,CFD-Q4-2005,CWD-Q1-2010,CWD-Q2-2010,CWD-Q3-2010,CWD-Q4-2010,CWD-Q1-2005,CWD-Q2-2005,CWD-Q3-2005,CWD-Q4-2005,WW-Q1-2010,WW-Q2-2010,WW-Q3-2010,WW-Q4-2010,WW-Q1-2005,WW-Q2-2005,...,FD-04-05-2010,FD-04-05-2005,FD-05-05-2010,FD-05-05-2005,FD-06-05-2010,FD-06-05-2005,FD-07-05-2010,FD-07-05-2005,FD-08-05-2010,FD-08-05-2005,FD-09-05-2010,FD-09-05-2005,FD-10-05-2010,FD-10-05-2005,FD-11-05-2010,FD-11-05-2005,FD-12-05-2010,FD-12-05-2005,FD-01-15-2010,FD-01-15-2005,FD-02-15-2010,FD-02-15-2005,FD-03-15-2010,FD-03-15-2005,FD-04-15-2010,FD-04-15-2005,FD-05-15-2010,FD-05-15-2005,FD-06-15-2010,FD-06-15-2005,FD-07-15-2010,FD-07-15-2005,FD-08-15-2010,FD-08-15-2005,FD-09-15-2010,FD-09-15-2005,FD-10-15-2010,FD-10-15-2005,soil_types,climate_zones
0,0,0,4383640,123.291667,53.541667,CHN,3918.1,2819.2,53.75,123.25,27.0,29.0,9.0,17.0,57.0,33.0,14.0,36.0,90.0,61.0,0.0,65.0,90.0,65.0,1.0,64.0,3.0,5.0,6.0,4.0,2.0,3.0,7.0,5.0,3.0,0.0,1.0,2.0,0.0,0.0,...,10.0,10.0,7.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,3.0,7.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,0.0,10.0,10.0,40,9
1,1,3,4393629,122.375000,53.458333,CHN,3119.3,2028.5,53.25,122.25,27.0,22.0,9.0,17.0,34.0,49.0,10.0,13.0,90.0,61.0,1.0,65.0,90.0,65.0,1.0,64.0,3.0,5.0,4.0,4.0,1.0,2.0,8.0,3.0,2.0,0.0,2.0,1.0,0.0,0.0,...,10.0,10.0,6.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,10.0,10.0,40,9
2,2,8,4403648,123.958333,53.375000,CHN,3230.1,2337.9,53.25,123.75,27.0,22.0,9.0,16.0,48.0,49.0,11.0,11.0,90.0,61.0,1.0,65.0,90.0,65.0,2.0,61.0,3.0,5.0,4.0,3.0,2.0,3.0,5.0,3.0,1.0,1.0,2.0,0.0,0.0,0.0,...,10.0,10.0,8.0,7.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,10.0,10.0,40,6
3,3,9,4403649,124.041667,53.375000,CHN,3119.3,2010.2,53.25,124.25,20.0,22.0,9.0,17.0,48.0,49.0,11.0,37.0,90.0,61.0,0.0,65.0,90.0,65.0,1.0,61.0,3.0,5.0,4.0,3.0,2.0,3.0,5.0,3.0,1.0,1.0,3.0,0.0,0.0,0.0,...,10.0,10.0,8.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,10.0,10.0,40,6
4,4,10,4413637,123.041667,53.291667,CHN,3918.1,2819.2,53.25,123.25,27.0,29.0,9.0,16.0,27.0,49.0,8.0,14.0,90.0,61.0,1.0,73.0,90.0,65.0,1.0,61.0,3.0,5.0,4.0,4.0,2.0,2.0,7.0,3.0,3.0,0.0,1.0,1.0,0.0,0.0,...,10.0,10.0,8.0,8.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,4.0,7.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,10.0,10.0,40,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427053,427053,832697,7660984,-98.041667,26.208333,USA,6148.3,6278.7,26.25,-97.25,16.0,23.0,33.0,16.0,12.0,48.0,24.0,19.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,11.0,13.0,10.0,3.0,2.0,6.0,7.0,1.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76,-997
427054,427054,832720,7670981,-98.291667,26.125000,USA,5713.1,5832.9,26.25,-97.25,16.0,23.0,33.0,16.0,12.0,48.0,24.0,19.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,11.0,13.0,10.0,3.0,2.0,6.0,7.0,1.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76,-997
427055,427055,832721,

In [0]:
data = dataset.copy()

## Data processing ## 

We now use Ed's script to process the data somewhat. Here we exclude the 10 day features.

In [7]:
# Create a custom data set adding in the features you want. It reads an already computed 
# dataset but you can then remove features to see the effect on score.
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import random

random_seed = 42
#random.seed(a=random_seed)
#np.random_seed(random_seed)

read_file_path = f"head_of_soils_recommendations_MGM.csv"

# Only read a sample fraction of the data, speeds things up, but you'll get worse performance.
# Use 1.0 as the sample fraction to get the full data set
sample_fraction = 0.9
data = pd.read_csv(read_file_path, skiprows=lambda i: i>0 and random.random() > sample_fraction)

limit_change_in_yield = True
limit = 0.5

include_coordinates = True
include_2005_maize = True
include_seasonal = True
include_ten_day = False
include_2005_climate = True
include_soil_type = True
include_growing_zones = True
include_climate_zones = True

# Remove datapoints where yield changes by more than 50%. This is unlikely to be
# due to changes in climate change.

if limit_change_in_yield:
    data = data[abs(data['maize_a_2010'] - data['maize_a_2005']) / data['maize_a_2005'] < limit]

# Choose a subset of your features! (must already exist in original file)
ten_day_features = [
    'BEDD',
    'R20mm',
    'DTR'
]

seasonal_features = [
    'CDD',
    'CFD',
    'CWD',
    'WW',
    'WSDI',
    'CSDI'
]

features = []

if include_growing_zones:
    growing_zones = [
        'Inland water bodies',
        'Subtropics - summer rainfall',
        'Subtropics - winter rainfall',
        'Temperature - continental',
        'Temperature - oceanic',
        'Temperature - subcontinental',
        'Tropics'
    ]
    
    
if include_soil_type:
    features = features + ['soil_types']

        
if include_climate_zones:
    features = features + ['climate_zones']

if include_2005_climate:
    years = ['2010', '2005']
else:
    years = ['2010']

if include_seasonal:
    seasons = ['Q1', 'Q2', 'Q3', 'Q4']
    years = ['2010', '2005']
    for feature in seasonal_features:
        for season in seasons:
            for year in years:
                features.append(f'{feature}-{season}-{year}')

if include_ten_day:
    days = ['05', '15', '25']
    years = ['2010', '2005']
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    for feature in ten_day_features:
        for day in days:
            for month in months:
                for year in years:
                    features.append(f'{feature}-{month}-{day}-{year}')

if include_coordinates:
    coordinates = [
        'x',
        'y'
    ]
    features = features + coordinates
    
if include_2005_maize:
    features = features + ['maize_a_2005']

target = 'maize_a_2010'
data = data.dropna()

X = data[features]
X.drop('x', axis=1, inplace=True) #drop x but keep y so we can tell something about the hemispheres and seasons


y = data[[target]]
train_set_x, test_set_x, train_set_y, test_set_y = train_test_split(X, y, test_size=0.2, random_state=random_seed)
del X
del y
del data

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
# drop features we don't want

#dataset = dataset.drop(['x','y','iso3_2005','Unnamed: 0', 'Unnamed: 0.1','Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'delta_maiz', 'maiz_percent_change'], axis=1)

In [0]:
train_set_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243624 entries, 56647 to 152031
Data columns (total 52 columns):
soil_types       243624 non-null int64
climate_zones    243624 non-null int64
CDD-Q1-2010      243624 non-null float64
CDD-Q1-2005      243624 non-null float64
CDD-Q2-2010      243624 non-null float64
CDD-Q2-2005      243624 non-null float64
CDD-Q3-2010      243624 non-null float64
CDD-Q3-2005      243624 non-null float64
CDD-Q4-2010      243624 non-null float64
CDD-Q4-2005      243624 non-null float64
CFD-Q1-2010      243624 non-null float64
CFD-Q1-2005      243624 non-null float64
CFD-Q2-2010      243624 non-null float64
CFD-Q2-2005      243624 non-null float64
CFD-Q3-2010      243624 non-null float64
CFD-Q3-2005      243624 non-null float64
CFD-Q4-2010      243624 non-null float64
CFD-Q4-2005      243624 non-null float64
CWD-Q1-2010      243624 non-null float64
CWD-Q1-2005      243624 non-null float64
CWD-Q2-2010      243624 non-null float64
CWD-Q2-2005      243624 non-

In [0]:
#dataset.dropna(inplace=True)

### Create train test split ### 

In [0]:
#from sklearn.model_selection import train_test_split

#train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

#train_set_x = train_set.drop("maiz_a_2010",axis=1)
#train_set_y = train_set["maiz_a_2010"]

#test_set_x = test_set.drop("maiz_a_2010",axis=1)
#test_set_y = test_set["maiz_a_2010"]

### One-hot encode categorical features ###


In [0]:
categorical_variables = ["climate_zones",
                         "soil_types"
                       
                         ]

In [0]:
categories=[]

for i in categorical_variables:
  a = list(dataset[i].unique())
  categories.append(a)


In [0]:
df_num = train_set_x.copy()

for i in categorical_variables:
  df_num.drop(i, axis=1, inplace=True)

Below pipeline didn't work one time for the onehotencoding so it's been commented out and replaced.

In [0]:
# from sklearn.base import BaseEstimator, TransformerMixin

# class DataFrameSelector(BaseEstimator, TransformerMixin):
#   def __init__(self,attribute_names):
#     self.attribute_names = attribute_names
#   def fit(self, X, y=None):
#     return self
#   def transform(self, X):
#     return X[self.attribute_names].values

# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.preprocessing import OneHotEncoder

# num_attributes = list(df_num)
# cat_attribs = categorical_variables

# num_pipeline = Pipeline([
#                          ('selector', DataFrameSelector(num_attributes)),
# ])

# cat_pipeline = Pipeline([
#                          ('selector', DataFrameSelector(cat_attribs)),
#                          ('one_hot_encoder', OneHotEncoder(categories = categories)),
                         
# ])

# full_pipeline = FeatureUnion(transformer_list=[
#                   ("num_pipeline",num_pipeline),
#                   ("cat_pipeline", cat_pipeline)
# ])

In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
                        ("selector", OneHotEncoder(), [0,1])], 
                        remainder = 'passthrough')

#0 and 1 refer to the categorical column names - these need to be changed so they are selected automatically and not manually as done here

In [0]:
train_set_x_prepared = ct.fit_transform(train_set_x)


In [0]:
train_set_x_prepared.shape

(243624, 154)

In [0]:
test_set_x_prepared = ct.transform(test_set_x)


In [0]:
pd.DataFrame(train_set_x_prepared)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,13.0,9.0,89.0,92.0,16.0,12.0,0.0,0.0,1.0,0.0,7.0,11.0,2.0,3.0,15.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.208333,3941.3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,14.0,10.0,8.0,6.0,6.0,8.0,7.0,8.0,4.0,0.0,1.0,5.0,3.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-28.458333,5159.6
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,28.0,0.0,0.0,17.0,12.0,3.0,5.0,3.0,4.0,6.0,7.0,4.0,3.0,2.0,1.0,1.0,1.0,8.0,1.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.375000,10113.1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,44.0,8.0,34.0,41.0,49.0,16.0,46.0,64.0,1.0,0.0,6.0,2.0,1.0,2.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.625000,2111.6
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0,2.0,79.0,62.0,10.0,18.0,0.0,0.0,0.0,0.0,9.0,1.0,0.0,2.0,0.0,0.0,15.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.125000,1788.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,4.0,0.0,0.0,1.0,1.0,9.0,4.0,7.0,4.0,5.0,4.0,3.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,6.0,0.0,0.0,0.0,0.0,39.208333,11711.3
243620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,15.0,21.0,17.0,22.0,0.0,0.0,18.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,12.0,6.0,0.0,8.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,-17.541667,1373.9
243621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,11.0,24.0,19.0,19.0,23.0,24.0,24.0,5.0,0.0,4.0,4.0,0.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,19.375000,1809.0
243622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,16.0,26.0,23.0,30.0,2.0,1.0,9.0,6.0,0.0,5.0,2.0,10.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.291667,1596.8


### Rough feature engineering ### 

1. Exclude features with low variance

In [0]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.2)
train_set_x_reduced = selector.fit_transform(train_set_x_prepared)
test_set_x_reduced = selector.transform(test_set_x_prepared)

In [0]:
train_set_x_reduced.shape

(243695, 51)

In [0]:
selector.get_support() #these are the features which are kept ... need to work out which ones they are

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

2. Or/and train a simple extra trees model and select most important features

In [0]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel

simple_model = ExtraTreesRegressor(n_jobs=-1, verbose=1, n_estimators=10)
simple_model = simple_model.fit(train_set_x_prepared, train_set_y)






/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.2min finished


In [0]:
simple_model.score(train_set_x_prepared, train_set_y) # we are able to fit to the training set at least

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    1.2s finished


0.99981701893788

In [0]:
simple_model.score(test_set_x_prepared, test_set_y) 

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.3s finished


0.9668285578750894

The below selects features based on feature importance but only gives 4 features when selecting based on mean, so I've not done it here.

In [0]:
# selector = SelectFromModel(simple_model, prefit=True)
# train_set_x_reduced = selector.transform(train_set_x_prepared)
# test_set_x_reduced = selector.transform(test_set_x_prepared)

#### Feature engineering without one-hot encoding ####

In [0]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.2)
train_set_x_reduced_2 = selector.fit_transform(train_set_x)
test_set_x_reduced_2 = selector.transform(test_set_x)

# Model exploration # 

## Which models can at least fit the training data? ##

### Random Forests ###


For this example we can run without doing the feature selection above.

In [0]:
from sklearn.ensemble import RandomForestRegressor

rnd_reg = RandomForestRegressor(n_estimators=20, max_leaf_nodes=30, n_jobs=-1)
rnd_reg.fit(train_set_x_prepared,train_set_y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=16,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

Can we overfit to the training set? Does the model have enough sophisitication to overfit to the trainin set...

In [0]:
rnd_reg.score(train_set_x_prepared,train_set_y)

0.9106381929671832

It could overfit if I train with enough max leaf nodes.




#### H2O implementation ####

In [0]:
import h2o

In [0]:
from h2o.estimators.random_forest import H2ORandomForestEstimator
rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1, categorical_encoding='enum')
rf_fit1.train(x=train_set_x_reduced_2, y=train_set_y, training_frame=train)


### Extra trees ###


In [25]:
from sklearn.ensemble import ExtraTreesRegressor

extra_trees = ExtraTreesRegressor(n_jobs=-1, verbose=1, n_estimators=10)
extra_trees = extra_trees.fit(train_set_x_prepared, train_set_y)

extra_trees.score(train_set_x_prepared,train_set_y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.3min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    1.1s finished


0.9997909847775208

In [26]:
extra_trees.score(test_set_x_prepared,test_set_y)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.3s finished


0.9675384416331634

We can fit to the training set at least.

### Extreme Boosting ###

In [0]:
!pip3 install xgboost

In [0]:
import xgboost as xgb


In [0]:
xgbmodel = xgb.XGBRegressor(n_jobs=-1, verboxity=2, random_state=42, n_estimators=100, objective="reg:squarederror", max_depth=20,)

In [0]:
xgbmodel.fit(train_set_x_prepared,train_set_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=20, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1, verboxity=2)

In [0]:
from sklearn.metrics import r2_score



In [0]:
r2_score(train_set_y, xgbmodel.predict(train_set_x_prepared))

0.9975928970862603

In [0]:
train_set_y

,maize_a_2010
140076,5778.3
233620,7638.7
321377,1568.3
125005,5132.8
157050,2242.6
...,...
149710,2614.2
332415,618.8
164446,953.7
184376,1051.1


## Randomised Grid Search of parameter space ## 

In [0]:
from sklearn.model_selection import RandomizedSearchCV

In [0]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform

### Extra Trees ###

In [0]:
parameter_grid = {

  'n_estimators': sp_randint(1,50),
  'max_depth':  sp_randint(1,40),
  'min_samples_split': uniform(0.01,0.199)

}

In [0]:
from sklearn.ensemble import ExtraTreesRegressor

extra_trees_search = ExtraTreesRegressor()

In [0]:
random_search = RandomizedSearchCV(extra_trees_search, parameter_grid, cv=5, n_jobs=-1, random_state=42)

In [20]:
random_search.fit(train_set_x_prepared,train_set_y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                                 criterion='mse',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=None,
                                                 oob_score=False...
                   para

In [21]:
random_search.cv_results_

{'mean_fit_time': array([ 72.47455616, 205.2609889 ,  70.10993247, 229.11446571,
         10.60546746, 180.70490422,  43.5544241 , 138.95962658,
        243.65751877,  62.53788862]),
 'mean_score_time': array([0.13923707, 0.3873415 , 0.17662063, 0.42622657, 0.05305338,
        0.2810226 , 0.09270654, 0.21287332, 0.28956146, 0.09422545]),
 'mean_test_score': array([0.86505246, 0.87791248, 0.90934645, 0.89583211, 0.84559105,
        0.86720167, 0.60017713, 0.89855338, 0.89839234, 0.85528205]),
 'param_max_depth': masked_array(data=[39, 8, 19, 24, 24, 24, 2, 33, 25, 28],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[0.16851205438518635, 0.12913303835521028,
                    0.029895008247782573, 0.07640801361666534,
                    0.13952680611682175, 0.17565608551928394,
                    0.04618316847421303, 0.07054

In [22]:
random_search.best_estimator_

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=19, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=0.029895008247782573,
                    min_weight_fraction_leaf=0.0, n_estimators=11, n_jobs=None,
                    oob_score=False, random_state=None, verbose=0,
                    warm_start=False)

In [23]:
random_search.best_score_

0.909346448145319

In [24]:
random_search.scorer_

<function sklearn.metrics._scorer._passthrough_scorer>

In [27]:
extra_trees3 = ExtraTreesRegressor(n_jobs=-1, verbose=1, n_estimators=10)
extra_trees3 = extra_trees.fit(train_set_x_prepared, train_set_y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.4min finished


In [29]:
extra_trees3.score(test_set_x_prepared,test_set_y)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.3s finished


0.9678790055348738

In [30]:
extra_trees3

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=10, n_jobs=-1, oob_score=False,
                    random_state=None, verbose=1, warm_start=False)

### Extreme Boosting ###